In [5]:
#import dependencies
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import os

In [4]:
#set variables for data files
mouse_metadata = os.path.join("Resources", "mouse_metadata.csv")
study_files = os.path.join("Resources", "study_results.csv")

#read the csv files
mouse_metadata = pd.read_csv(mouse_metadata)
study_files = pd.read_csv(study_files)
mouse_metadata.head()

#combine the data sets into one
full_data_df = pd.merge(mouse_metadata, study_files, on="Mouse ID")

pandas.core.frame.DataFrame